In [12]:
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

url = "http://proceedings.mlr.press/v70/"

# If the folder does not exist, create one automatically
folder_location = './webscraping/'
if not os.path.exists(folder_location):
    os.mkdir(folder_location)

response = requests.get(url)
print(response)
soup = BeautifulSoup(response.text, "html.parser")
for link in soup.select("a[href$='.pdf']"):
    print("goes here")
    #Name the pdf files using the last portion of each link
    filename = os.path.join(folder_location, link['href'].split('/')[-1])
    with open(filename, 'wb') as f:
        f.write(requests.get(urljoin(url, link['href'])).content)
        


<Response [200]>
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goes here
goe

In [ ]:
pdf2txt.py hu17a-supp.pdf 